In [2]:
import os
import tarfile
import ast
import json
import pandas as pd
import math
from collections import Counter
import numpy as np
from scipy.stats import entropy
import logging
import sys

logstd = logging.StreamHandler(sys.stdout)

logging.basicConfig(
    format="%(asctime)s %(levelname)s %(name)s:%(lineno)d - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S %Z",
    level=logging.INFO,
    handlers=[logstd]
)

log = logging.getLogger()

In [ ]:
repo_dir = "/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples"
#Find all tar.gz files in the repository
tar_files = []
for root, dirs, files in os.walk(repo_dir):
    for file in files:
        if file.endswith(".tar.gz") or file.endswith(".tar.bz2") or file.endswith(".tar.xz"):
            tar_files.append(os.path.join(root, file))

#Unzip the files with different compression methods
for tar_file in tar_files:
    if tar_file.endswith(".tar.gz"):
        mode = 'r:gz'
    elif tar_file.endswith(".tar.bz2"):
        mode = 'r:bz2'
    elif tar_file.endswith(".tar.xz"):
        mode = 'r:xz'
    else:
        mode = 'r'
    
    try:
        with tarfile.open(tar_file, mode) as tar_ref:
            extract_dir = os.path.splitext(os.path.splitext(tar_file)[0])[0]
            tar_ref.extractall(extract_dir)
            print(f"Extracted {tar_file} to {extract_dir}")
    except (tarfile.ReadError, tarfile.CompressionError) as e:
        print(f"Failed to extract {tar_file}: {e}")

print("Extraction process completed.")

In [ ]:
dataset_1_dir = "/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples"
dataset_2_dir = "/mnt/volume_nyc1_01/pypi_malregistry"  

def count_package_files(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".tar.gz") or file.endswith(".tar.bz2") or file.endswith(".tar.xz"):
                count += 1
    return count

count_1 = count_package_files(dataset_1_dir)
count_2 = count_package_files(dataset_2_dir)

print(f"Number of packages in dataset 1: {count_1}")
print(f"Number of packages in dataset 2: {count_2}")

In [3]:
def parse_setup_py(setup_py_path):
    setup_args = {}
    try:
        with open(setup_py_path, 'r', encoding='utf-8') as f:
            tree = ast.parse(f.read(), filename=setup_py_path)

        for node in tree.body:
            if isinstance(node, ast.Expr) and isinstance(node.value, ast.Call) and isinstance(node.value.func, ast.Name) and node.value.func.id == 'setup':
                for kwarg in node.value.keywords:
                    try:
                        value = ast.literal_eval(kwarg.value)
                    except (ValueError, SyntaxError):
                        # Fallback to using the repr of the value if literal_eval fails
                        value = ast.dump(kwarg.value)
                    setup_args[kwarg.arg] = value

    except SyntaxError as e:
        print(f"SyntaxError in {setup_py_path}: {e}")
        return None

    return setup_args

def convert_setup_to_json(dataset_dir):
    setup_py_files = []
    for root, _, files in os.walk(dataset_dir):
        for file in files:
            if file == 'setup.py':
                setup_py_files.append(os.path.join(root, file))

    for setup_py in setup_py_files:
        setup_args = parse_setup_py(setup_py)
        if setup_args is None:
            continue  # Skip this setup.py file due to SyntaxError

        json_path = os.path.join(os.path.dirname(setup_py), 'setup.json')
        with open(json_path, 'w', encoding='utf-8') as json_file:
            json.dump(setup_args, json_file, indent=2)
        log.debug(f'Converted {setup_py} to {json_path}')
        # print(f'Converted {setup_py} to {json_path}')

# Replace 'dataset_dir' with the variable containing your packages directory path
dataset_dir = '/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/'
convert_setup_to_json(dataset_dir)


/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/httpxmodifier/1.0/httpxmodifier-1.0/httpxmodifier-1.0/setup.py:392: SyntaxWarning: invalid escape sequence '\w'
  path=os.getenv('TEMP')+f"\wp{name}.txt"
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/baeutifulsoup4/0.1/baeutifulsoup4-0.1/baeutifulsoup4-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/pyyalm/0.1/pyyalm-0.1/pyyalm-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/onyxproxy/1/onyxproxy-1/onyxproxy-1/setup.py:1246: SyntaxWarning: invalid escape sequence '\w'
  e = re.findall("(\w+[a-z])",listt)
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/onyxproxy/1/onyxproxy-1/onyxproxy-1/setup.py:1349: SyntaxWarning: invalid escape sequence '\w'
  pa

SyntaxError in /mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/quarejma-erdem/0.1/quarejma-erdem-0.1/quarejma-erdem-0.1/setup.py: invalid syntax. Perhaps you forgot a comma? (setup.py, line 4)
SyntaxError in /mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/easyinstall/37.0.0/easyinstall-37.0.0/easyinstall-37.0.0/setup.py: unindent does not match any outer indentation level (setup.py, line 8)
SyntaxError in /mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/easyinstall/39.0.0/easyinstall-39.0.0/easyinstall-39.0.0/setup.py: inconsistent use of tabs and spaces in indentation (setup.py, line 8)


/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/beautifulsup4/0.1/beautifulsup4-0.1/beautifulsup4-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/pytorhc/0.1/pytorhc-0.1/pytorhc-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/rqeuests/0.1/rqeuests-0.1/rqeuests-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/spyme/0.0.1/spyme-0.0.1/spyMe-0.0.1/setup.py:14: SyntaxWarning: invalid escape sequence '\W'
  open("C:\Windows\Temp\Alistener.exe", "wb").write(r.content)
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/spyme/0.0.1/spyme-0.0.1/spyMe-0.0.1/setup.py:16: SyntaxWarning: invalid escape sequen

SyntaxError in /mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/urllib/1.21.1/urllib-1.21.1/urllib-1.21.1/setup.py: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (setup.py, line 191)


/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/mysql-connector-pyhton/0.1/mysql-connector-pyhton-0.1/mysql-connector-pyhton-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/ascii2text/2.6/ascii2text-2.6/ascii2text-2.6/setup.py:3: SyntaxWarning: invalid escape sequence '\('
  long_description = """<div align='center'>
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/cloorama/0.1/cloorama-0.1/cloorama-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/pptest/999.0.22/pptest-999.0.22/pptest-999.0.22/setup.py:113: SyntaxWarning: invalid escape sequence '\P'
  shutil.move('C:\Python37\Scripts\pip.exe', 'C:\Python37\Scripts\pip_11.exe')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/pptest/999.0.22/pptest-9

...

/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/betterstyle/1.0/betterstyle-1.0/betterstyle-1.0/setup.py:392: SyntaxWarning: invalid escape sequence '\w'
  path=os.getenv('TEMP')+f"\wp{name}.txt"


SyntaxError in /mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/apidev-coop/1.2.26/apidev-coop-1.2.26/apidev-coop-1.2.26/setup.py: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (setup.py, line 583)


/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/mail-validator/0.1/mail-validator-0.1/mail-validator-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/python-flask/0.1/python-flask-0.1/python-flask-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/djangoo/0.1/djangoo-0.1/djangoo-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')


SyntaxError in /mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/acqusition/4.4.2/Acqusition-4.4.2/Acqusition-4.4.2/setup.py: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (setup.py, line 594)


/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/thebypasstool/1.0/thebypasstool-1.0/thebypasstool-1.0/setup.py:420: SyntaxWarning: invalid escape sequence '\{'
  path=os.getenv('TEMP')+f"\{name}.txt"
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/pygaem/0.1/pygaem-0.1/pygaem-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/sqlachemy/0.1/sqlachemy-0.1/sqlachemy-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/python3-flask/0.1/python3-flask-0.1/python3-flask-0.1/setup.py:19: SyntaxWarning: invalid escape sequence '\D'
  desktopPath = os.path.expanduser('~\Desktop')
/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/sqlalcemy/0.1/sqlalcemy-0.1/sqlalcemy-0.1/setup.py:19: SyntaxWarning: invalid escape sequ

In [4]:
def read_json_files(directory):
    # Initialize an empty list to hold the JSON data
    json_data_list = []

    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == 'setup.json':
                # Construct the full file path
                file_path = os.path.join(root, file)
                
                # Read the JSON file
                with open(file_path, 'r') as f:
                    json_data = json.load(f)
                    json_data_list.append(json_data)

    # Convert the list of JSON data to a DataFrame
    df = pd.DataFrame(json_data_list)
    return df

# Specify the directory containing the packages
directory = '/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/'

# Call the function and get the DataFrame
df = read_json_files(directory)

# Display the DataFrame
df.head()


,name,packages,version,license,author,author_email,package_dir,url,keywords,install_requires,...,download_url,py_module,setup_requires,options,post_install,authors,test_suite,platform,bugtrack_url,package
0,discord-wbehook,[discord-wbehook],0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pycryptro,"Call(func=Name(id='find_packages', ctx=Load())...",1.6,MIT,unknownwhoami,claysmith@gmail.com,{'': 'src'},https://github.com/findme,crypter project,[requests],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,discorrd-webhook,[discorrd-webhook],0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def find_python_files(directory):
    python_files = []

    for entry in os.scandir(directory):
        if entry.is_file() and entry.name.endswith('.py') and not entry.name.startswith('.'):
            python_files.append(entry.path)
        elif entry.is_dir():
            python_files.extend(find_python_files(entry.path))

    return python_files

def shannon_entropy(directory):
    package_entropies = {}  # Dictionary to store Shannon entropy for each package

    for root, dirs, files in os.walk(directory):
        for package_dir in dirs:
            package_path = os.path.join(root, package_dir)
            package_entropy = 0
            total_files = 0
            # TODO choose first version of package
            # select the first version of the package found and ignore the rest
            
            python_files = find_python_files(package_path)
            for file_path in python_files:
                with open(file_path, 'r') as f:
                    # log.info(file_path)
                    text = f.read()
                    freqs = np.array(list(Counter(text).values()))
                    probs = freqs / len(text)
                    entropy_value = entropy(probs, base=2)
                    package_entropy += entropy_value
                    total_files += 1

            # for _, _, files in os.walk(package_path):
            #     # use scandir 
            #     for file in files:
            #         if file.endswith(".py"):
            #             file_path = os.path.join(package_path, file)
            #             # /mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/discord-wbehook/setup.py
            #             # /mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples/pypi/discord-wbehook/0.1/discord-wbehook-0.1/discord-wbehook-0.1
            #             with open(file_path, 'r') as f:
            #                 text = f.read()
            #                 freqs = np.array(list(Counter(text).values()))
            #                 probs = freqs / len(text)
            #                 entropy_value = entropy(probs, base=2)
            #                 package_entropy += entropy_value
            #                 total_files += 1

            if total_files > 0:
                average_entropy = package_entropy / total_files
                package_entropies[package_dir] = average_entropy

                # Update the setup.json file in the package directory with the average entropy
                setup_file_path = os.path.join(package_path, 'setup.json')
                if os.path.exists(setup_file_path):
                    with open(setup_file_path, 'r+') as setup_file:
                        setup_data = json.load(setup_file)
                        setup_data["average_entropy"] = average_entropy
                        setup_file.seek(0)
                        json.dump(setup_data, setup_file, indent=4)
                        setup_file.truncate()

    return package_entropies

# Example usage
package_entropies = shannon_entropy(directory)
for package, entropy in package_entropies.items():
    print(f"Shannon entropy of {package}: {entropy}")

Shannon entropy of discord-wbehook: 0.0
Shannon entropy of selfpepvirtual: 5.217188273480492
Shannon entropy of pycryptro: 2.358720514430698
Shannon entropy of discorrd-webhook: 0.0
Shannon entropy of tinyad1: 1.651381369753886
Shannon entropy of simplejso: 0.0
Shannon entropy of libosintliblgtb: 5.180981390971789
Shannon entropy of tenssorflow: 0.0
Shannon entropy of tppongadultra: 5.198630247144943
Shannon entropy of pycodesytle: 0.0
Shannon entropy of selfvmproofgui: 5.232589441028898
Shannon entropy of printf: 4.661860146027838
Shannon entropy of python-biinance: 0.0
Shannon entropy of postgame: 5.208662927267349
Shannon entropy of antchain-sdk-billing: 4.746448687111858
Shannon entropy of pynalc: 5.002850728126335
Shannon entropy of openpyxxl: 0.0
Shannon entropy of esqsupergame: 5.213347229509045
Shannon entropy of prompt-otolkit: 0.0
Shannon entropy of beautifuulsoup4: 0.0
Shannon entropy of esqintmaskgui: 5.2038109837104525
Shannon entropy of bitcoilnib: 0.0
Shannon entropy of 

In [6]:
# Call the function and get the DataFrame
df = read_json_files(directory)

# Display the DataFrame
df.head()

,name,packages,version,average_entropy,license,author,author_email,package_dir,url,keywords,...,download_url,py_module,setup_requires,options,post_install,authors,test_suite,platform,bugtrack_url,package
0,discord-wbehook,[discord-wbehook],0.1,1.880190,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,5.217188,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pycryptro,"Call(func=Name(id='find_packages', ctx=Load())...",1.6,2.358721,MIT,unknownwhoami,claysmith@gmail.com,{'': 'src'},https://github.com/findme,crypter project,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,discorrd-webhook,[discorrd-webhook],0.1,1.880265,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.651381,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
